<a href="https://colab.research.google.com/github/SwastikGorai/automatic-octo-umbrella/blob/main/asl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
!pip install tensorflow-gpu==2.4.0
!pip install kaggle
!pip install opencv-python


In [ ]:
# Replace "YOUR_API_KEY" with your actual API key
# If it throws error, run cell again
!echo '{"username":"YOUR_USERNAME","key":"YOUR_API_KEY"}' > /root/.kaggle/kaggle.json

# Download the ASL dataset
!kaggle datasets download -d grassknoted/asl-alphabet

# Unzip the downloaded dataset
!unzip asl-alphabet.zip


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os

# Load the data
X = []
y = []

# Load the images and labels from the dataset
for directory in os.listdir('asl_alphabet_train'):
  for filename in os.listdir(f'asl_alphabet_train/{directory}'):
    for ff in os.listdir(f'asl_alphabet_train/{directory}/{filename}'):
      image = cv2.imread(f'asl_alphabet_train/{directory}/{filename}/{ff}')
      print(image)
      if image is not None:
        image = cv2.resize(image, (64, 64))
        X.append(image)
        y.append(directory)

In [8]:
# Convert the data to NumPy arrays
X = np.array(X)
y = np.array(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# To check Nullity of variables
print(X_train, X_val, y_train, y_val)

In [9]:
# Define the model
model = tf.keras.models.Sequential()

# Add the first convolutional layer
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Add the second convolutional layer
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Add the third convolutional layer
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Add a flattening layer
model.add(tf.keras.layers.Flatten())


In [10]:
# Add the first fully connected layer
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))

# Add the second fully connected layer
model.add(tf.keras.layers.Dense(26, activation='softmax'))

In [11]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
from sklearn.preprocessing import LabelEncoder

# Encode the labels
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_val = encoder.transform(y_val)


In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)

print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')


In [15]:
# Save the model
model.save('model.h5')

In [ ]:
# Save the model in the SavedModel format
tf.saved_model.save(model, 'saved_model')

In [ ]:
# Save the model in .tflite format
!tflite_convert \
  --saved_model_dir=saved_model \
  --output_file=model.tflite

